In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, mean_squared_error
import pandas as pd
#import tensorflow as tf

In [2]:
# Import and read the csv's
##supply data
key_data = pd.read_csv("../data/old/00-key.csv")
sku_data = pd.read_csv("../data/old/01-SKU.csv")
dist_data = pd.read_csv("../data/old/02-DIST.csv")

##demand data
posa_data = pd.read_csv("../data/old/03-POS-A.csv")
posb_data = pd.read_csv("../data/old/03-POS-B.csv")
sect_data = pd.read_csv("../data/old/04-SECT.csv")

matrix_data = pd.read_csv("../data/old/05-MATRIX.csv")

In [3]:
#display table (CHECKING IF IT RAN CORRECTLY)
#dist_data.head()
posa_data.head()

,BLDG_ID,SqFtCd,SqFt_est,POS_Landuse,sectorid,POS_StateClass,POS_StateClassCd,center_x,center_y
0,317,UNDER 5000SF,1777.219873,NaN,1,RESIDENTIAL,A1,-95.3526,29.76921
1,630,UNDER 5000SF,2179.123515,NaN,1,RESIDENTIAL,A1,-95.3526,29.76901
2,1132,UNDER 5000SF,2127.308579,NaN,1,RESIDENTIAL,A1,-95.3525,29.76926
3,1277,UNDER 5000SF,4427.082666,NaN,1,RESIDENTIAL,A1,-95.3545,29.76754
4,1328,UNDER 5000SF,1467.869286,NaN,1,RESIDENTIAL,A1,-95.3526,29.76871


In [4]:
# Drop NaN
key_data_df = key_data.dropna()
sku_data_df = sku_data.dropna()
dist_data_df = dist_data.dropna()
posa_data_df = posa_data.dropna()
posb_data_df = posb_data.dropna()
sect_data_df = sect_data.dropna()
matrix_data_df = matrix_data.dropna()



In [5]:
# Merge the key and sku dataframes for supply df
combined_supply_df = pd.merge(key_data_df, sku_data_df, on='schema_id')
combined_supply_df.head()

,schema_id,name_desc,avg_stock,avg_price_x,avg_mktshare,SKU_ID,sku_name,avg_price_y,supply_desc,distribution,production
0,S01,Soybeans,802075,0.22,176456,1001020,soy nuts,4.49,food packaging,commercial,unfermented
1,S01,Soybeans,802075,0.22,176456,1001021,edemame,5.26,food packaging,commercial,unfermented
2,S02,Soybean meal,683667,0.19,129897,1001018,tofu,2.00,food manufacturing,commercial,unfermented
3,S02,Soybean meal,683667,0.19,129897,1001019,soymilk,3.49,food manufacturing,commercial,unfermented
4,S02,Soybean meal,683667,0.19,129897,1001022,tempeh,7.37,food manufacturing,commercial,fermented


In [6]:
# Merge dist with all of supply df
combined_supply_df = pd.merge(combined_supply_df, dist_data_df, on='schema_id')
combined_supply_df.head()

,schema_id,name_desc,avg_stock,avg_price_x,avg_mktshare,SKU_ID,sku_name,avg_price_y,supply_desc_x,distribution,production,CUST_ID,CUST__NAME,supply_desc_y,CUST_ADDRESS
0,S01,Soybeans,802075,0.22,176456,1001020,soy nuts,4.49,food packaging,commercial,unfermented,2001005,Anu Resources,food packaging,"11757 Katy Freeway, Suite 1300, Houston 77079 ..."
1,S01,Soybeans,802075,0.22,176456,1001020,soy nuts,4.49,food packaging,commercial,unfermented,2001006,Al Hakeem Co,food packaging,9318 Lynchester Dr Houston 77083 Texas United ...
2,S01,Soybeans,802075,0.22,176456,1001020,soy nuts,4.49,food packaging,commercial,unfermented,2001007,Lien Hoa B. Inc,food packaging,7611 Summer Glen Ln Houston 77072 United States
3,S01,Soybeans,802075,0.22,176456,1001020,soy nuts,4.49,food packaging,commercial,unfermented,2001011,Cronusweb Corporation,food packaging,2425 West Loop South Suite 200 Houston 77027 T...
4,S01,Soybeans,802075,0.22,176456,1001020,soy nuts,4.49,food packaging,commercial,unfermented,2001014,Kavantures LLC,food packaging,3032 South Camino Lagos Grand Prairie 75054 Te...


In [7]:
# Merge the POS A and POS B dataframes for demand ds
combined_demand_df = pd.merge(posa_data_df, posb_data_df, on='BLDG_ID')
combined_demand_df.head()

,BLDG_ID,SqFtCd_x,SqFt_est,POS_Landuse,sectorid,POS_StateClass,POS_StateClassCd,center_x,center_y,POS_Contact,...,POS_NAICS,POS_North American Industry Classification,POS_TYPE_focus,POS_Revenue,Owner_Ethnicity,Owner_Type,Owner_Established,Owner_CreditScore,SqFtCd_y,SqFt_range
0,528,UNDER 5000SF,2317.560900,COMMERCIAL,2,COMMERCIAL,F1,-95.3519,29.76013,IRMA G GALVAN,...,72251117,FULL-SERVICE RESTAURANTS,RESTAURANTS,"$500,000-$1 MILLION",HISPANIC,FIRM,1990,B+,3.0,"2,500 - 4,999"
1,707,UNDER 5000SF,4948.907364,COMMERCIAL,4,COMMERCIAL,F1,-95.3632,29.74908,LISA JUE,...,72251117,FULL-SERVICE RESTAURANTS,RESTAURANTS,"$500,000-$1 MILLION",CHINESE,FIRM,1991,B+,3.0,"2,500 - 4,999"
2,177,UNDER 5000SF,4861.121116,COMMERCIAL,10,COMMERCIAL,F1,-95.3648,29.76156,ALLAN LEVINE,...,72251117,FULL-SERVICE RESTAURANTS,RESTAURANTS,$2.5-5 MILLION,JEWISH,FIRM,2012,B+,1.0,"1 - 1,499"
3,214,UNDER 5000SF,2753.035979,COMMERCIAL,0,COMMERCIAL,F1,-95.3772,29.75309,ELIZABETH KNOX,...,72241001,DRINKING PLACES ALCOHOLIC BEVERAGES,BAR,"$500,000-$1 MILLION",SCOTTISH (SCOTCH),FIRM,2004,B+,2.0,"1,500 - 2,499"
4,559,UNDER 5000SF,4349.422191,COMMERCIAL,0,COMMERCIAL,F1,-95.3775,29.75268,JOSH HARRIS,...,72241001,DRINKING PLACES ALCOHOLIC BEVERAGES,BAR,$1-2.5 MILLION,ENGLISH,FIRM,2002,A,4.0,"5,000 - 9,999"


In [9]:
# Merge sect with all of demand df so we have it on the data what sector id name they are with 
combined_demand_df = pd.merge(combined_demand_df, sect_data_df, on='sectorid')
combined_demand_df.head()

,BLDG_ID,SqFtCd_x,SqFt_est,POS_Landuse,sectorid,POS_StateClass,POS_StateClassCd,center_x,center_y,POS_Contact,...,POS_TYPE_focus,POS_Revenue,Owner_Ethnicity,Owner_Type,Owner_Established,Owner_CreditScore,SqFtCd_y,SqFt_range,sectorid_name_x,sectorid_name_y
0,528,UNDER 5000SF,2317.560900,COMMERCIAL,2,COMMERCIAL,F1,-95.3519,29.76013,IRMA G GALVAN,...,RESTAURANTS,"$500,000-$1 MILLION",HISPANIC,FIRM,1990,B+,3.0,"2,500 - 4,999",Minute Maid Park,Minute Maid Park
1,707,UNDER 5000SF,4948.907364,COMMERCIAL,4,COMMERCIAL,F1,-95.3632,29.74908,LISA JUE,...,RESTAURANTS,"$500,000-$1 MILLION",CHINESE,FIRM,1991,B+,3.0,"2,500 - 4,999",Toyota Center,Toyota Center
2,177,UNDER 5000SF,4861.121116,COMMERCIAL,10,COMMERCIAL,F1,-95.3648,29.76156,ALLAN LEVINE,...,RESTAURANTS,$2.5-5 MILLION,JEWISH,FIRM,2012,B+,1.0,"1 - 1,499",Market Square,Market Square
3,214,UNDER 5000SF,2753.035979,COMMERCIAL,0,COMMERCIAL,F1,-95.3772,29.75309,ELIZABETH KNOX,...,BAR,"$500,000-$1 MILLION",SCOTTISH (SCOTCH),FIRM,2004,B+,2.0,"1,500 - 2,499",Downtown Periphery,Downtown Periphery
4,559,UNDER 5000SF,4349.422191,COMMERCIAL,0,COMMERCIAL,F1,-95.3775,29.75268,JOSH HARRIS,...,BAR,$1-2.5 MILLION,ENGLISH,FIRM,2002,A,4.0,"5,000 - 9,999",Downtown Periphery,Downtown Periphery


In [10]:
# Describe the supply new df
combined_supply_df.describe()

,avg_stock,avg_price_x,avg_mktshare,SKU_ID,avg_price_y,CUST_ID
count,227.000000,227.000000,227.000000,2.270000e+02,227.000000,2.270000e+02
mean,239575.722467,3.079824,72552.383260,1.001013e+06,12.115242,2.001022e+06
std,326686.508202,2.997312,48023.638918,7.956962e+00,20.023863,1.185245e+01
min,3185.000000,0.190000,37213.000000,1.001000e+06,0.670000,2.001000e+06
25%,5390.000000,0.190000,40519.000000,1.001005e+06,4.490000,2.001012e+06
50%,15925.000000,2.540000,40519.000000,1.001014e+06,6.320000,2.001023e+06
75%,683667.000000,6.900000,129897.000000,1.001019e+06,7.370000,2.001029e+06
max,802075.000000,13.660000,176456.000000,1.001026e+06,90.000000,2.001043e+06


In [11]:
# Describe the demand new df
combined_demand_df.describe()

,BLDG_ID,SqFt_est,sectorid,center_x,center_y,Latitude,Longitude,POS_SELSIC,POS_NAICS,Owner_Established,SqFtCd_y
count,10.00000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,1.000000e+01,10.000000,10.000000
mean,732.10000,3472.381349,2.700000,-95.365770,29.755002,29.754990,-95.365780,543965.200000,6.814437e+07,2005.100000,2.900000
std,409.08528,1055.447572,4.321779,0.010086,0.004885,0.004878,0.010098,117870.113203,1.297601e+07,9.314863,1.523884
min,177.00000,2317.560900,0.000000,-95.377500,29.749080,29.749100,-95.377500,208501.000000,3.121400e+07,1990.000000,1.000000
25%,535.75000,2541.670882,0.000000,-95.376325,29.752147,29.752150,-95.376325,581208.000000,7.224100e+07,2002.000000,2.000000
50%,645.50000,3357.409565,0.000000,-95.364000,29.752885,29.752850,-95.364000,581208.000000,7.225112e+07,2005.000000,3.000000
75%,973.00000,4255.496708,3.500000,-95.357075,29.759498,29.759475,-95.357075,581277.750000,7.225112e+07,2012.000000,3.750000
max,1342.00000,4948.907364,11.000000,-95.351900,29.761980,29.762000,-95.351900,581301.000000,7.225112e+07,2016.000000,6.000000


In [12]:
# look at schema_id value count for binning
print(combined_supply_df['schema_id'].value_counts())

B01    90
S02    60
B02    56
S01    14
B03     5
S03     2
Name: schema_id, dtype: int64


In [13]:
# look at bldg_id value count for binning
print(combined_demand_df['BLDG_ID'].value_counts())

528     1
707     1
177     1
214     1
559     1
859     1
1011    1
1340    1
1342    1
584     1
Name: BLDG_ID, dtype: int64


In [15]:
# Convert categorical data(selected columns) to numeric in supply df (using get dummies)
supply_cols = ['supply_desc_x','supply_desc_y','distribution','production']
supply_dummies = pd.get_dummies(combined_supply_df, columns=supply_cols)
supply_dummies.head()

,schema_id,name_desc,avg_stock,avg_price_x,avg_mktshare,SKU_ID,sku_name,avg_price_y,CUST_ID,CUST__NAME,CUST_ADDRESS,supply_desc_x_food manufacturing,supply_desc_x_food packaging,supply_desc_y_food manufacturing,supply_desc_y_food packaging,distribution_commercial,distribution_wholesale,production_fermented,production_processed,production_unfermented
0,S01,Soybeans,802075,0.22,176456,1001020,soy nuts,4.49,2001005,Anu Resources,"11757 Katy Freeway, Suite 1300, Houston 77079 ...",0,1,0,1,1,0,0,0,1
1,S01,Soybeans,802075,0.22,176456,1001020,soy nuts,4.49,2001006,Al Hakeem Co,9318 Lynchester Dr Houston 77083 Texas United ...,0,1,0,1,1,0,0,0,1
2,S01,Soybeans,802075,0.22,176456,1001020,soy nuts,4.49,2001007,Lien Hoa B. Inc,7611 Summer Glen Ln Houston 77072 United States,0,1,0,1,1,0,0,0,1
3,S01,Soybeans,802075,0.22,176456,1001020,soy nuts,4.49,2001011,Cronusweb Corporation,2425 West Loop South Suite 200 Houston 77027 T...,0,1,0,1,1,0,0,0,1
4,S01,Soybeans,802075,0.22,176456,1001020,soy nuts,4.49,2001014,Kavantures LLC,3032 South Camino Lagos Grand Prairie 75054 Te...,0,1,0,1,1,0,0,0,1


In [16]:
# Convert categorical data(selected columns) to numeric in demand df (using get dummies)
dem_cols = ['POS_StateClass','POS_North American Industry Classification','POS_Revenue']
demand_dummies =pd.get_dummies(combined_demand_df, columns=dem_cols)
demand_dummies.head()


,BLDG_ID,SqFtCd_x,SqFt_est,POS_Landuse,sectorid,POS_StateClassCd,center_x,center_y,POS_Contact,POS_Name,...,POS_StateClass_COMMERCIAL,POS_StateClass_COMMUNITY,POS_North American Industry Classification_DISTILLERIES,POS_North American Industry Classification_DRINKING PLACES ALCOHOLIC BEVERAGES,POS_North American Industry Classification_FULL-SERVICE RESTAURANTS,POS_Revenue_$1-2.5 MILLION,POS_Revenue_$2.5-5 MILLION,POS_Revenue_$5-10 MILLION,"POS_Revenue_$500,000-$1 MILLION","POS_Revenue_LESS THAN $500,000"
0,528,UNDER 5000SF,2317.560900,COMMERCIAL,2,F1,-95.3519,29.76013,IRMA G GALVAN,IRMA'S RESTAURANT,...,1,0,0,0,1,0,0,0,1,0
1,707,UNDER 5000SF,4948.907364,COMMERCIAL,4,F1,-95.3632,29.74908,LISA JUE,CHINA GARDEN,...,1,0,0,0,1,0,0,0,1,0
2,177,UNDER 5000SF,4861.121116,COMMERCIAL,10,F1,-95.3648,29.76156,ALLAN LEVINE,CULTIVATED F+B,...,1,0,0,0,1,0,1,0,0,0
3,214,UNDER 5000SF,2753.035979,COMMERCIAL,0,F1,-95.3772,29.75309,ELIZABETH KNOX,KOMODO'S,...,1,0,0,1,0,0,0,0,1,0
4,559,UNDER 5000SF,4349.422191,COMMERCIAL,0,F1,-95.3775,29.75268,JOSH HARRIS,FRONT PORCH PUB,...,1,0,0,1,0,1,0,0,0,0


In [17]:
# Grabbing the matrix csv file 
matrix_data_df

,POS_TYPE_focus,S01,S02,S03,B01,B02,B03,Total
0,BAR,0.10,0.20,0.40,0.05,0.65,0.10,1.50
1,BEVERAGE STORES,0.00,0.00,0.10,0.00,0.00,0.00,0.10
2,BREWERY,0.00,0.00,0.10,0.00,0.00,0.05,0.15
3,CATERERS,0.25,0.25,0.25,0.25,0.25,0.25,1.50
4,CHAIN RESTAURANTS,0.00,0.10,0.40,0.65,0.00,0.00,1.15
5,ENTERTAINMENT,0.00,0.00,0.40,0.25,0.00,0.00,0.65
6,FOOD STORES,0.10,0.10,0.15,0.10,0.10,0.10,0.65
7,GENERAL STORES,0.10,0.10,0.10,0.10,0.00,0.10,0.50
8,HOTELS,0.10,0.85,0.10,0.25,0.10,0.10,1.50
9,PHARMACY STORES,0.00,0.00,0.05,0.00,0.00,0.05,0.10


In [18]:
# Convert the matrix(wide) df into a longer form
matrix_df = matrix_data_df.melt(id_vars=['POS_TYPE_focus'], var_name='schema_id', value_name='correlation')


In [19]:
# Create a merged demand df with matrix_df so it keeps only data from demand df (connects it together to build our model)
combined_demand_df = pd.merge(demand_dummies, matrix_df, on=['POS_TYPE_focus'], how='left')
combined_demand_df.head()

,BLDG_ID,SqFtCd_x,SqFt_est,POS_Landuse,sectorid,POS_StateClassCd,center_x,center_y,POS_Contact,POS_Name,...,POS_North American Industry Classification_DISTILLERIES,POS_North American Industry Classification_DRINKING PLACES ALCOHOLIC BEVERAGES,POS_North American Industry Classification_FULL-SERVICE RESTAURANTS,POS_Revenue_$1-2.5 MILLION,POS_Revenue_$2.5-5 MILLION,POS_Revenue_$5-10 MILLION,"POS_Revenue_$500,000-$1 MILLION","POS_Revenue_LESS THAN $500,000",schema_id,correlation
0,528,UNDER 5000SF,2317.5609,COMMERCIAL,2,F1,-95.3519,29.76013,IRMA G GALVAN,IRMA'S RESTAURANT,...,0,0,1,0,0,0,1,0,S01,0.25
1,528,UNDER 5000SF,2317.5609,COMMERCIAL,2,F1,-95.3519,29.76013,IRMA G GALVAN,IRMA'S RESTAURANT,...,0,0,1,0,0,0,1,0,S02,0.25
2,528,UNDER 5000SF,2317.5609,COMMERCIAL,2,F1,-95.3519,29.76013,IRMA G GALVAN,IRMA'S RESTAURANT,...,0,0,1,0,0,0,1,0,S03,0.25
3,528,UNDER 5000SF,2317.5609,COMMERCIAL,2,F1,-95.3519,29.76013,IRMA G GALVAN,IRMA'S RESTAURANT,...,0,0,1,0,0,0,1,0,B01,0.25
4,528,UNDER 5000SF,2317.5609,COMMERCIAL,2,F1,-95.3519,29.76013,IRMA G GALVAN,IRMA'S RESTAURANT,...,0,0,1,0,0,0,1,0,B02,0.25


In [21]:
# Get volume for sku
combined_demand_df['volume_sku'] = (
    (combined_demand_df['SqFt_est'] * combined_demand_df['correlation'])
    / (
        combined_demand_df['POS_Revenue_$1-2.5 MILLION'] * 100
        + combined_demand_df['POS_Revenue_$2.5-5 MILLION'] * 150
        + combined_demand_df['POS_Revenue_$5-10 MILLION'] * 200
        + combined_demand_df['POS_Revenue_$500,000-$1 MILLION'] * 50
        + combined_demand_df['POS_Revenue_LESS THAN $500,000'] * 25
    )
)
combined_demand_df.head()

,BLDG_ID,SqFtCd_x,SqFt_est,POS_Landuse,sectorid,POS_StateClassCd,center_x,center_y,POS_Contact,POS_Name,...,POS_North American Industry Classification_DRINKING PLACES ALCOHOLIC BEVERAGES,POS_North American Industry Classification_FULL-SERVICE RESTAURANTS,POS_Revenue_$1-2.5 MILLION,POS_Revenue_$2.5-5 MILLION,POS_Revenue_$5-10 MILLION,"POS_Revenue_$500,000-$1 MILLION","POS_Revenue_LESS THAN $500,000",schema_id,correlation,volume_sku
0,528,UNDER 5000SF,2317.5609,COMMERCIAL,2,F1,-95.3519,29.76013,IRMA G GALVAN,IRMA'S RESTAURANT,...,0,1,0,0,0,1,0,S01,0.25,11.587804
1,528,UNDER 5000SF,2317.5609,COMMERCIAL,2,F1,-95.3519,29.76013,IRMA G GALVAN,IRMA'S RESTAURANT,...,0,1,0,0,0,1,0,S02,0.25,11.587804
2,528,UNDER 5000SF,2317.5609,COMMERCIAL,2,F1,-95.3519,29.76013,IRMA G GALVAN,IRMA'S RESTAURANT,...,0,1,0,0,0,1,0,S03,0.25,11.587804
3,528,UNDER 5000SF,2317.5609,COMMERCIAL,2,F1,-95.3519,29.76013,IRMA G GALVAN,IRMA'S RESTAURANT,...,0,1,0,0,0,1,0,B01,0.25,11.587804
4,528,UNDER 5000SF,2317.5609,COMMERCIAL,2,F1,-95.3519,29.76013,IRMA G GALVAN,IRMA'S RESTAURANT,...,0,1,0,0,0,1,0,B02,0.25,11.587804


In [25]:
# Check shape of supply df for feature matrix
supply_dummies.shape

(227, 20)

In [24]:
# Check shape of demand df for feature matrix
combined_demand_df.shape


(70, 37)

In [55]:
# Grab supply features
supply_features = supply_dummies[['schema_id','avg_stock','avg_price_x','avg_price_y', 'avg_mktshare']]

In [56]:
# Merge demand and supply features on schema_id
mega_df = pd.merge(combined_demand_df, supply_features, on='schema_id')
mega_df.head()

,BLDG_ID,SqFtCd_x,SqFt_est,POS_Landuse,sectorid,POS_StateClassCd,center_x,center_y,POS_Contact,POS_Name,...,POS_Revenue_$5-10 MILLION,"POS_Revenue_$500,000-$1 MILLION","POS_Revenue_LESS THAN $500,000",schema_id,correlation,volume_sku,avg_stock,avg_price_x,avg_price_y,avg_mktshare
0,528,UNDER 5000SF,2317.5609,COMMERCIAL,2,F1,-95.3519,29.76013,IRMA G GALVAN,IRMA'S RESTAURANT,...,0,1,0,S01,0.25,11.587804,802075,0.22,4.49,176456
1,528,UNDER 5000SF,2317.5609,COMMERCIAL,2,F1,-95.3519,29.76013,IRMA G GALVAN,IRMA'S RESTAURANT,...,0,1,0,S01,0.25,11.587804,802075,0.22,4.49,176456
2,528,UNDER 5000SF,2317.5609,COMMERCIAL,2,F1,-95.3519,29.76013,IRMA G GALVAN,IRMA'S RESTAURANT,...,0,1,0,S01,0.25,11.587804,802075,0.22,4.49,176456
3,528,UNDER 5000SF,2317.5609,COMMERCIAL,2,F1,-95.3519,29.76013,IRMA G GALVAN,IRMA'S RESTAURANT,...,0,1,0,S01,0.25,11.587804,802075,0.22,4.49,176456
4,528,UNDER 5000SF,2317.5609,COMMERCIAL,2,F1,-95.3519,29.76013,IRMA G GALVAN,IRMA'S RESTAURANT,...,0,1,0,S01,0.25,11.587804,802075,0.22,4.49,176456


In [57]:
# Grab demand features
demand_features = combined_demand_df[['SqFt_est','correlation','sectorid']]

In [62]:
# Merge mega df and matrix on schema_id
mega_df = pd.merge(mega_df, matrix_data_df, left_on='POS_TYPE_focus', right_on='POS_TYPE_focus', how='left')
mega_df.head()

,BLDG_ID,SqFtCd_x,SqFt_est,POS_Landuse,sectorid,POS_StateClassCd,center_x,center_y,POS_Contact,POS_Name,...,B02_x,B03_x,Total_x,S01_y,S02_y,S03_y,B01_y,B02_y,B03_y,Total_y
0,528,UNDER 5000SF,2317.5609,COMMERCIAL,2,F1,-95.3519,29.76013,IRMA G GALVAN,IRMA'S RESTAURANT,...,0.25,0.25,1.5,0.25,0.25,0.25,0.25,0.25,0.25,1.5
1,528,UNDER 5000SF,2317.5609,COMMERCIAL,2,F1,-95.3519,29.76013,IRMA G GALVAN,IRMA'S RESTAURANT,...,0.25,0.25,1.5,0.25,0.25,0.25,0.25,0.25,0.25,1.5
2,528,UNDER 5000SF,2317.5609,COMMERCIAL,2,F1,-95.3519,29.76013,IRMA G GALVAN,IRMA'S RESTAURANT,...,0.25,0.25,1.5,0.25,0.25,0.25,0.25,0.25,0.25,1.5
3,528,UNDER 5000SF,2317.5609,COMMERCIAL,2,F1,-95.3519,29.76013,IRMA G GALVAN,IRMA'S RESTAURANT,...,0.25,0.25,1.5,0.25,0.25,0.25,0.25,0.25,0.25,1.5
4,528,UNDER 5000SF,2317.5609,COMMERCIAL,2,F1,-95.3519,29.76013,IRMA G GALVAN,IRMA'S RESTAURANT,...,0.25,0.25,1.5,0.25,0.25,0.25,0.25,0.25,0.25,1.5


In [82]:
combined_demand_df['volume_sku'].shape

(70,)

Train and Start the Model - CODE IS WRITTEN BUT NOT COMPLETE WITH NEEDED INFORMATION STILL NEEDED

In [81]:
# define y & X for training
y = combined_demand_df['volume_sku']
X = pd.concat([supply_features, demand_features, combined_demand_df.drop(['POS_TYPE_focus'], axis=1)], axis=1)


TypeError: cannot concatenate object of type '<class 'numpy.ndarray'>'; only Series and DataFrame objs are valid

In [72]:
# Import numpy & reshape X
# import numpy as np
# X = np.reshape(X, (70,20))

In [80]:
# Train test with the correct variables
# X = X.transpose()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

ValueError: Found input variables with inconsistent numbers of samples: [227, 70]

HAVE NOT BEEN ABLE TO CONITINUE YET

In [ ]:
# import and start on the Random Forest model
from sklearn.ensemble import RandomForestClassifier

# n estimator - creates a model with 3 trees in the forest
# max depth - each tree will have 3 levels
rfc = RandomForestClassifier(n_estimators=3, max_depth=2, random_state=42)

In [ ]:
# use fit RandomForestClassifier
rfc.fit(X_train, y_train)
# Predict the test set labels
y_pred = rfc.predict(X_test)

In [ ]:
# Visualize each of the trees and how they dividing the data
# Import `tree` module
from sklearn import tree

# The name of each column
features = X.columns.values
# The name of each class 
classes = ['0', '1', '2'] 

for estimator in rfc.estimators_:
    print(estimator)
    plt.figure(figsize=(12,6))
    tree.plot_tree(estimator,
                   feature_names=features,
                   class_names=classes,
                   fontsize=8, 
                   filled=True, 
                   rounded=True)
    plt.show()

In [ ]:
# Receive classification report and confusion matrix
from sklearn.metrics import classification_report, confusion_matrix

cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d').set_title('Maternal risks confusion matrix (0 = low risk, 1 = medium risk, 2 = high risk)')

print(classification_report(y_test,y_pred))

In [ ]:
# We'll most likely have to retest the model for better results

In [ ]:
# Calculate Mean Abs. Error, Mean Squared Error, Root Mean Squared Root Error
from sklearn.metrics import mean_absolute_error, mean_squared_error

print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, y_pred)))
